In [ ]:
import pandas as pd
import gensim
from gensim import corpora,models
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel



In [ ]:
swearwords = """cumbubble
fuck
fuck you
shitbag
shit
piss off
asshole
dickweed
cunt
son of a bitch
fuck trumpet
bastard
bitch
damn
bollocks
bugger
cocknose
bloody hell
knobhead
choad
bitchtits
crikey
rubbish
pissflaps
shag
wanker
talking the piss
twat
arsebadger
jizzcock
cumdumpster
shitmagnet
scrote
twatwaffle
thundercunt
dickhead
shitpouch
jizzstain
nonce
pisskidney
wazzock
cumwipe
fanny
bellend
pisswizard
knobjockey
cuntpuddle
dickweasel
quim
bawbag
fuckwit
tosspot
cockwomble
twat face
cack
flange
clunge
dickfucker
fannyflaps
wankface
shithouse
gobshite
jizzbreath
todger
nutsack
motherfucker
fuck
fucking
dick
pussy
ha
word
yeah
cause
somebody
someone
gotta
everyone
question
sex
everybody
question
bit
idea
""".split('\n')


In [ ]:
stopwords = set(stopwords.words('english'))
stopwords.update(set(swearwords))

In [ ]:
df = pd.read_csv('comedy_cleaned.csv', sep='\t')

In [ ]:
df['year'].value_counts()

In [ ]:
# import re
#
# def remove_non_printable_chars(text):
#     return re.sub(r'[^\x20-\x7E]', '', text)
#
# df['text'] = df['text'].apply(remove_non_printable_chars)

In [ ]:
docs = df['text'].tolist()

In [ ]:
# create a dictionary
years = {
    'Pre-2000': [1900, 2000],
    '2000-2010': [2000, 2010],
    '2010-2020': [2010, 2020],
    'Post-2020': [2020, 2100]
}

In [ ]:
# find rows in df where year is between 2000 and 2010
for (key, value) in years.items():
    df1 = df.loc[(df['year'] >= value[0]) & (df['year'] < value[1])]
    years[key].append(df1['text'].tolist())


In [ ]:
for (key, value) in years.items():
    print(key, len(value[2]))

In [ ]:
from nltk import pos_tag

def extract_nouns(tokenized_sentences):
    noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']


    nouns  = [[x for x, tag in pos_tag(sentence) if tag in noun_tags] for sentence in tokenized_sentences]

    return nouns

In [ ]:
topics = []
for (key, value) in years.items():
    docs = value[2]
    for idx in range(len(docs)):
        tokenizer = RegexpTokenizer(r'\w+')
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    docs = [[token for token in doc if not token.isnumeric()] for doc in docs]
    docs = [[token for token in doc if token not in stopwords] for doc in docs]
    docs = [[token for token in doc if len(token) > 1] for doc in docs]
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

    docs = extract_nouns(docs)

    bigram = Phrases(docs,min_count = 10)
    for idx in range(len(docs)):
        for token in bigram[docs[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                docs[idx].append(token)
    dictionary = Dictionary(docs)
    dictionary.filter_extremes(no_below=10, no_above=0.8)

    corpus = [dictionary.doc2bow(doc) for doc in docs]
    word_counts = [[(dictionary[id], count) for id, count in line] for line in corpus]
    sort_token = sorted(dictionary.items(),key=lambda k:k[0], reverse = False)
    unique_token = [token for (ID,token) in sort_token]
    matrix = gensim.matutils.corpus2dense(corpus,num_terms=len(dictionary),dtype = 'int')

    matrix = matrix.T #transpose the matrix
    matrix_df = pd.DataFrame(matrix, columns=unique_token)

    num_topics = 1
    chunksize = 2000
    #chenksize is the number of documents to be used in each training chunk.
    passes = 20 #The number of model training in the whole corpus
    iterations = 100 #Number of iterations of per document
    eval_every = 1  # Don't evaluate model perplexity, takes too much time.

    temp = dictionary[0]  # This is only to "load" the dictionary.
    id2word = dictionary.id2token

    lda = LdaModel(
        corpus=corpus,
        id2word=id2word,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )
    print(lda)
    # for i,topic in lda.print_topics(10):
    #     print(f'Top 10 words for topic #{i+1}:')
    #     print(topic)
    #     print('\n')

    def get_top_topics(lda_model, num_topics=10, num_words=10):
        top_topics = lda_model.show_topics(num_topics=num_topics, num_words=num_words, formatted=False)
        topics = []

        for topic in top_topics:
            topic_words = [word for word, _ in topic[1]]
            topics.append(topic_words)

        return topics

    top_topics = get_top_topics(lda, num_topics=1, num_words=10)
    topics.append(top_topics[0])


In [ ]:
lda.show_topics()

In [ ]:
import pandas as pd
pd.DataFrame(topics).T